In [ ]:
import os
import torch
import scipy.stats as stats
import numpy as np

# CODE FILES HERE
from models.vae.vae import Encoder, Decoder, Vae, MODEL_NAME
from solver import Solver
from directories import Directories
from dataloader import DataLoader
from plots import plot_losses, plot_gaussian_distributions, plot_rl_kl, plot_latent_space, \
plot_latent_manifold, plot_faces_grid, plot_faces_samples_grid
from auxiliary import get_latent_spaces

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')
# VARIABLES HERE
load_model = False

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:
# Choose the dataset and tune hyperparameters here!
dataset = "MNIST"

batch_size = 128
optimizer = torch.optim.Adam

if dataset == "MNIST":
    epochs = 5
    hidden_dim = 500
    z_dim = 2
    beta = 4 if z_dim == 2 else 4
    batch_norm = False
    lr_scheduler = torch.optim.lr_scheduler.StepLR
    step_config = {
        "step_size" : 300,
        "gamma" : 0.1 # or 0.75
    }
    optim_config = {
        "lr": 1e-3,
        "weight_decay": None
    }
elif dataset == "LFW":
    epochs = 30
    hidden_dim = 700
    z_dim = 40
    beta = 1 if z_dim == 2 else 1
    batch_norm = True
    lr_scheduler = torch.optim.lr_scheduler.StepLR
    step_config = {
        "step_size" : 30,
        "gamma" : 0.1
    }
    optim_config = {
        "lr": 1e-2,
        "weight_decay": None
    }
elif dataset == "FF":
    epochs = 1500
    hidden_dim = 200
    z_dim = 2
    beta = 1
    batch_norm = True
    lr_scheduler = torch.optim.lr_scheduler.StepLR
    step_config = {
        "step_size" : 300,
        "gamma" : 0.1
    }
    optim_config = {
        "lr": 1e-2,
        "weight_decay": None
    }

In [ ]:
directories = Directories(MODEL_NAME, dataset, z_dim, False)
data_loader = DataLoader(directories, batch_size, dataset)
model = Vae(data_loader.input_dim, hidden_dim, z_dim, beta, batch_norm)
solver = Solver(model, data_loader, optimizer, epochs, optim_config, step_config=step_config, lr_scheduler=lr_scheduler)
solver.main()

In [ ]:
# Insert name of model here if want to load a model, e.g. solver.save_model_dir + "/VAE_MNIST_train_loss=151.39_z=2.pt"
#solver = torch.load(solver.save_model_dir + "/VAE_MNIST_train_loss=97.15_z=20.pt")
#solver.model.eval()
#load_model = True

In [ ]:
# Plotting train and test losses for all epochs
plot_losses(solver, solver.train_loss_history["train_loss_acc"], solver.test_loss_history)

In [ ]:
# Plotting the gaussian of z space and some metrics about the space
plot_gaussian_distributions(solver, len(solver.train_loss_history["train_loss_acc"]))

In [ ]:
# Monitoring the reconstruction loss (likelihood lower bound) and KL divergence
DEBUG = 1
if DEBUG:
    for epoch, train_loss, test_loss, rl, kl in zip(solver.train_loss_history["epochs"], \
                             solver.train_loss_history["train_loss_acc"], \
                             solver.test_loss_history, \
                             solver.train_loss_history["recon_loss_acc"], \
                             solver.train_loss_history["kl_diverg_acc"]):
        print("epoch: {}, train_loss: {:.2f}, test_loss: {:.2f}, recon. loss: {:.2f}, KL div.: {:.2f}".format(
            epoch, train_loss, test_loss, rl, kl))
        print("overfitting: {:.2f}".format(abs(test_loss-train_loss)))
plot_rl_kl(solver, solver.train_loss_history["recon_loss_acc"], solver.train_loss_history["kl_diverg_acc"])

In [ ]:
# visualize q(z) (latent space z)
if solver.model.z_dim == 2:
    z_space, _, labels = get_latent_spaces(solver)
    if solver.data_loader.with_labels:
        plot_latent_space(solver, z_space, var="z", title="classes", labels=labels.tolist())
    else:
        plot_latent_space(solver, z_space, var="z")
else:
    print("Plot of latent space not possible as dimension of z is not 2")

In [ ]:
# Visualizations of learned data manifold for generative models with two-dimensional latent space
if solver.model.z_dim == 2 and not load_model:
    if solver.data_loader.dataset == "MNIST":
        n = 20
        grid_x = grid_y = np.linspace(-3, 3, n)
        plot_latent_manifold(solver, "bone", grid_x, grid_y, n)
    if solver.data_loader.dataset == "LFW" or solver.data_loader.dataset == "FF":
        n = 10
        grid_x = grid_y = np.linspace(-3, 3, n)
        plot_latent_manifold(solver, "gray", grid_x, grid_y, n, fig_size=(10, 8))
else:
    print("Plot is not possible as dimension of z is not 2 or model is load")

In [ ]:
# plots real faces and in grid samples
if solver.data_loader.dataset == "LFW" or solver.data_loader.dataset == "FF":
    n = 225
    n_cols = 15
    plot_faces_grid(n, n_cols, solver)
    plot_faces_samples_grid(n, n_cols, solver)

In [ ]:
last_train_loss = solver.train_loss_history["train_loss_acc"][-1]
torch.save(solver, solver.data_loader.directories.result_dir + "/model_VAE_" + solver.data_loader.dataset + "_train_loss=" + "{0:.2f}".format(last_train_loss) + "_z=" + str(solver.model.z_dim) + ".pt")